# Nova formulação

Uma nova formulação a partir do artigo

*Duxbury, Lavor, Liberti, Salles-Neto, Unassigned distance geometry and molecular conformation problems, Journal of Global Optimization, v.83, pp: 73-82, 2022.*

A ideia é substituir a função objetivo do modelo (3) no artigo acima por:
$$
    \min \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \left( \sum_{k=1}^{m} a_{ij}^k \big\vert \Vert x_i - x_j \Vert_2 - d_k \big\vert \right),
$$
onde $x_i = (x_{i,1}, x_{i,2}, x_{i,3})^\mathsf{T}$, $i = 1, 2, \ldots, n$.

In [ ]:
%pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

## Preprocessamento

In [ ]:
import numpy as np

# Lista de distâncias
n = 4
d = np.sort(
    np.array(
        [
            1.5,
            2.5244,
            3.8724,
            1.5,
            2.5244,
            1.5,
        ]
    )
)
m = len(d)

alfa = 0.1


Sejam
$$
    D = \max_{k=1,\ldots, m} \{ d_k \} \quad\text{e}\quad S = \sum_{k=1}^m d_k.
$$

In [ ]:
D = d.max()
S = d.sum()


## Modelo

### Variáveis

In [ ]:
model = gp.Model("UnassignedDistance")
model.setParam(GRB.Param.NonConvex, 2)

# Variável com as coordenadas
x = {
    (i, l): model.addVar(name=f"x_{i}_{l}", vtype=GRB.CONTINUOUS)
    for i in range(n)
    for l in range(3)
}

# Variável de decisão (distância k é referente ao par i, j.)
a = {
    (i, j, k): model.addVar(name=f"a_{i}_{j}_{k}", vtype=GRB.BINARY)
    for i in range(n - 1)
    for j in range(i + 1, n)
    for k in range(m)
}

# Variável z (distância entre os átomos i e j se distância k é referente ao par i, j. Zero em caso contrário.)
z = {
    (i, j, k): model.addVar(name=f"z_{i}_{j}_{k}", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
    for k in range(m)
}

# Variável w (distância entre os átomos i e j)
w = {
    (i, j): model.addVar(name=f"w_{i}_{j}", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
}


### Função objetivo

O novo modelo:
$$
    \text{(NP):} \quad \min \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \left( \sum_{k=1}^{m} z_{ijk} \right) - S,
$$

In [ ]:
model.setObjective(
    gp.quicksum(
        z[i, j, k] for i in range(n - 1) for j in range(i + 1, n) for k in range(m)
    )
    - S,
    GRB.MINIMIZE,
)


### Restrições

O problema está sujeito a:
$$
\begin{aligned}
    &\text{(C1):} & z_{ijk}^2 &= a_{ij}^k \sum_{l=1}^3 ( x_{i,l} - x_{j,l} )^2 \\
    &\text{(C2):} & z_{ijk} &\leq a_{ij}^k D \\
    &\text{(C2$^\prime$):} & z_{ijk} &\geq -a_{ij}^k D \\
    &\text{(C3):} & z_{ijk} &\leq d_k + (1-a_{ij}^k) D \\
    &\text{(C4):} & z_{ijk} &\geq d_k - (1-a_{ij}^k) D \\
\end{aligned}
$$
para $i = 1, 2, \ldots, n−1,\; j = i+1, i+2, \ldots, n,\; k = 1, 2, \ldots, m$

In [ ]:
for i in range(n - 1):
    for j in range(i + 1, n):

        model.addConstr(
            w[i, j] ** 2 == gp.quicksum((x[i, l] - x[j, l]) ** 2 for l in range(3)),
            name=f"w_{i}_{j}",
        )

        for k in range(m):
            # C1
            model.addConstr(
                z[i, j, k] ** 2 == a[i, j, k] * w[i, j],
            )
            # C2
            model.addConstr(z[i, j, k] <= a[i, j, k] * D)
            # C2'
            model.addConstr(z[i, j, k] >= -a[i, j, k] * D)
            # C3
            model.addConstr(z[i, j, k] <= (d[k] + alfa) + (1 - a[i, j, k]) * D)
            # C4
            model.addConstr(z[i, j, k] >= (d[k] - alfa) - (1 - a[i, j, k]) * D)


Mantendo as restrições do modelo (3):
$$
\begin{aligned}
    &\text{(C5):} & \sum_{i=1}^{n-1} \sum_{j=1+1}^{n} a_{ij}^k &= 1 && k = 1, 2, \ldots, m, \\
    &\text{(C6):} & \sum_{k=1}^{m} a_{ij}^k &\leq 1 && i = 1, 2, \ldots, n−1,\; j = i+1, i+2, \ldots, n,
\end{aligned}
$$

In [ ]:
# C5
c5 = model.addConstrs(
    gp.quicksum(a[i, j, k] for i in range(n - 1) for j in range(i + 1, n)) == 1
    for k in range(m)
)
# C6
c6 = model.addConstrs(
    gp.quicksum(a[i, j, k] for k in range(m)) <= 1
    for i in range(n - 1)
    for j in range(i + 1, n)
)


## Solução

In [ ]:
# Verify model formulation

model.write("unassigned_distance.lp")

# Run optimization engine

model.optimize()

## Resultados

In [ ]:
try:
    model.write("out.sol")
    model.printAttr('x')
except gp.GurobiError:
    pass